## Checking PyTorch and CUDA Availability

In this code cell, we import the PyTorch library and print out information about the availability of CUDA, the CUDA version, and the PyTorch version. This is useful for verifying that PyTorch is correctly installed and that it can utilize the GPU for computations.


In [1]:
import torch

print("CUDA Available:", torch.cuda.is_available()) 
print("CUDA Version:", torch.version.cuda)
print("PyTorch Version:", torch.__version__)


ModuleNotFoundError: No module named 'torch'

## Loading YOLOv5 Model

In this code cell, we load the YOLOv5 model from a local directory using PyTorch's hub module. After loading the model, a confirmation message is printed to indicate that the YOLOv5 model has been successfully loaded.


In [ ]:
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
YOLOV5_REPO = PROJECT_ROOT / "yolov5_src"
model = torch.hub.load(str(YOLOV5_REPO), 'yolov5n', source='local')
print("YOLOv5 model loaded successfully from", YOLOV5_REPO)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/anuragdineshrokade/Documents/YOLO-Object-Detection-and-Classification-for-ADAS/yolov5/yolov5/hubconf.py'

## Checking GPU Availability and Details

This cell checks the availability of CUDA and prints the number of GPUs available. For each available GPU, it also prints its name, helping to identify the GPUs accessible for running PyTorch operations.


In [ ]:

import torch

print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


## Training the YOLOv5 Model

This cell runs the training script for the YOLOv5 model. Ensure the `data.yaml` file is correctly placed in the `yolov5` directory or provide the correct path to it. The training is configured with an image size of 640, batch size of 16, and runs for 10 epochs. The training weights are initialized with `yolov5n.pt`, and the process runs on device 0 (usually the first GPU). The results are saved in the `runs/train` directory under the name `exp_model-n_img-640`.


In [ ]:
from pathlib import Path

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent
YOLOV5_REPO = PROJECT_ROOT / "yolov5_src"
DATA_YAML = PROJECT_ROOT / "dataset" / "data.yaml"
WEIGHTS = YOLOV5_REPO / "yolov5n.pt"
PROJECT_DIR = NOTEBOOK_DIR / "runs"
RUN_NAME = "exp_dummy"

!python {YOLOV5_REPO / 'train.py'} --img 640 --batch 4 --epochs 1 --data {DATA_YAML} --weights {WEIGHTS} --device cpu --project {PROJECT_DIR / 'train'} --name {RUN_NAME} --exist-ok

## Object Detection and Visualization with YOLOv5

This cell performs object detection on a set of test images using a custom-trained YOLOv5 model and visualizes the results with bounding boxes and labels.

1. **Load the YOLOv5 Model**: The model is loaded from the specified path where the custom weights are stored.
2. **Define a Function to Draw Bounding Boxes and Labels**: This function draws bounding boxes and labels on the detected objects in the image.
3. **Specify Paths**: Set the paths for the directory containing the test images and the directory where the processed images will be saved.
4. **Process Test Images**: 
    - List all image files in the test images directory.
    - For each image, apply the YOLOv5 model to detect objects.
    - Draw bounding boxes and labels on the detected objects.
    - Save the processed images to the output directory.
    - Optionally, display the processed images.

The bounding boxes and labels are drawn using OpenCV, and the processed images are saved to the specified output directory.


In [ ]:
import os
from pathlib import Path

import cv2
import numpy as np
import torch

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent
YOLOV5_REPO = PROJECT_ROOT / "yolov5_src"
RUN_DIR = NOTEBOOK_DIR / "runs" / "train" / "exp_dummy" / "weights"
WEIGHTS_PATH = (RUN_DIR / "best.pt") if (RUN_DIR / "best.pt").exists() else (RUN_DIR / "last.pt")
TEST_IMAGES_DIR = PROJECT_ROOT / "dataset" / "images" / "val"
OUTPUT_DIR = PROJECT_ROOT / "output_images_test_set"
OUTPUT_DIR.mkdir(exist_ok=True)

if not WEIGHTS_PATH.exists():
    raise FileNotFoundError(f"Expected weights not found in {RUN_DIR}")

if not TEST_IMAGES_DIR.exists():
    raise FileNotFoundError(f"Test images directory missing: {TEST_IMAGES_DIR}")

model = torch.hub.load(str(YOLOV5_REPO), 'custom', path=str(WEIGHTS_PATH), source='local', force_reload=True)


def draw_boxes(image, results):
    for pred in results.pred[0].detach().cpu().numpy():
        x1, y1, x2, y2, conf, cls = pred
        label = f"{model.names[int(cls)]} {conf:.2f}"
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image, (int(x1), int(y1) - 20), (int(x1) + w, int(y1)), (0, 255, 0), -1)
        cv2.putText(image, label, (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)


image_files = sorted([f for f in os.listdir(TEST_IMAGES_DIR) if f.lower().endswith(('.jpg', '.png'))])
print(f"Processing {len(image_files)} images from {TEST_IMAGES_DIR}")

for image_file in image_files:
    image_path = TEST_IMAGES_DIR / image_file
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Skipping unreadable file: {image_path}")
        continue

    results = model(image)
    draw_boxes(image, results)

    output_image_path = OUTPUT_DIR / image_file
    cv2.imwrite(str(output_image_path), image)
    print(f"Saved detections to {output_image_path}")

print(f"All processed images saved to {OUTPUT_DIR}")
